In [1]:
import keras.backend as K
from keras.layers import Convolution1D, MaxPooling1D, Dense, Flatten, Reshape
from keras.layers import Input
from keras.metrics import categorical_accuracy
from keras.models import Model, load_model
from keras.objectives import mean_squared_error, categorical_crossentropy

import os
import sys

sys.path.insert(0, os.path.abspath('../../Evolutron/'))

from evolutron.networks import Dedense, Upsampling1D, Deconvolution1D

Using Theano backend.


In [2]:
input_shape = (100, 20)
seq_length, alphabet = input_shape

# Input LayerRO
inp = Input(shape=input_shape, name='aa_seq')

In [3]:
n_filters = 30
filter_length = 10
n_conv_layers = 3
# Convolutional Layers
convs = [Convolution1D(filters=n_filters,
                       kernel_size=filter_length,
                       kernel_initializer='glorot_uniform',
                       activation='relu',
                       padding='same',
                       name='Conv1')(inp)]

for c in range(1, n_conv_layers):
    convs.append(Convolution1D(filters=n_filters,
                               kernel_size=filter_length,
                               kernel_initializer='glorot_uniform',
                               activation='relu',
                               padding='same',
                               name='Conv{}'.format(c + 1))(convs[-1]))  # maybe add L1 regularizer

In [4]:
convs[-1]._keras_shape

(None, 100, 30)

In [5]:
# Max-pooling
max_pool = MaxPooling1D(pool_size=seq_length)(convs[-1])
flat = Flatten()(max_pool)

In [6]:
print(max_pool._keras_shape)
print(flat._keras_shape)

(None, 1, 30)
(None, 30)


In [7]:
# Fully-Connected encoding layers
n_fc_layers=2
fc_enc = [Dense(n_filters,
                kernel_initializer='glorot_uniform',
                activation='sigmoid',
                name='FCEnc1')(flat)]

for d in range(1, n_fc_layers):
    fc_enc.append(Dense(n_filters,
                        kernel_initializer='glorot_uniform',
                        activation='sigmoid',
                        name='FCEnc{}'.format(d + 1))(fc_enc[-1]))

encoded = fc_enc[-1]  # To access if model for encoding needed

In [8]:
encoded._keras_shape

(None, 30)

In [9]:
fc_dec = [Dedense(encoded._keras_history[0],
                  activation='linear',
                  name='FCDec{}'.format(n_fc_layers))(encoded)]

for d in range(n_fc_layers - 2, -1, -1):
    fc_dec.append(Dedense(fc_enc[d]._keras_history[0],
                          activation='linear',
                          name='FCDec{}'.format(d + 1))(fc_dec[-1]))

In [10]:
fc_dec[-1]._keras_shape

(None, 30)

In [11]:
# Reshaping and unpooling
unflat = Reshape(max_pool._keras_shape[1:])(fc_dec[-1])
deconvs = [Upsampling1D(max_pool._keras_history[0].input_shape[1], name='Unpooling')(unflat)]

In [12]:
print(deconvs[-1]._keras_shape)

(None, 100, 30)


In [13]:
# Deconvolution
for c in range(n_conv_layers - 1, 0, -1):
    deconvs.append(Deconvolution1D(convs[c]._keras_history[0],
                                   activation='relu',
                                   name='Deconv{}'.format(c + 1))(deconvs[-1]))  # maybe add L1 regularizer

decoded = Deconvolution1D(convs[0]._keras_history[0],
                          apply_mask=False,
                          activation='sigmoid',
                          name='Deconv1')(deconvs[-1])

In [16]:
print(decoded._keras_shape)

(None, 100, 20)


In [20]:
coder = Model(inp, decoded)